# EE 379K: Lab 6

## Rohan Nagar and Wenyang Fu

In [1]:
import numpy as np

# Problem 1: K-Means

### Part 1

Implement the K-Means algorithm with a random initalization. Recall that there are two steps: given the centers, computing the clusters, and given the cluseters, computing the centers. As discussed in class, the second is straightforward. Describe how you do the first step.

### Part 2

Now implement K-Means++, i.e., the K-Means++ initalization.

### Part 3

Download the data `iris_data.csv` from Canvas. Plot Sepal Width vs. Sepal Length.

### Part 4

Use your K-Means++ algorithm to cluster the above two variables into 2, 3, 4, and 5 clusters.

# Problem 2: Spectral Clustering

Spectral clustering, as described in class, is a powerful and much-used algorithm. A very in-depth tutorial can be found here: http://www.cs.cmu.edu/~aarti/Class/10701/readings/Luxburg06_TR.pdf. In class we discussed the application of spectral clustering to a given graph. But we can also apply it to other data sets, by turning them into weighted graphs. This can be done in several ways, but roughly speaking, we convert by means of a similarity function: two nodes have an edge between them with weight equal to their similiarity. Read section 2 of the above tutorial on different ways to create the similiarity graph. Then read section 4 on the different graph clustering algorithms, especically focusing on the one by Ng, Jordan and Weiss (2002). This is the one we discussed in class.

### Part 1

Implement the Ng, Jordan and Weiss spectral clustering algorithm. You may use basic linear algebra functions, including SVD and eigenvalue decomposition, but do not use `sklearn.cluster.spectral.clustering` or anything similar. Of course, you can use your K-Means algorithm from above for the last step.

### Part 2

Write a program that generates N data points uniformly at random on two concentric rings, and then add two-dimensional Gaussian noise of variance $\sigma_1^2$ and $\sigma_2^2$ to each point. There are four parameters of interest here: the number of points, the difference in the radii of the two rings, the amount (the variance) of noise added to the internal ring, and the amount of noise added to the external ring.

### Part 3

Experiement with the 3 methods of creating similarity functions described in section 2 of the paper. Pick one that you like (clearly state which one), and fixing $N = 500$ and $\sigma_1 = \sigma_2 = 1$, and also the radius of the inner ring to be $1$, find the range of the outer ring radius for which spectral clustering, using your chosen similarity function, succeeds in clustering the two different rings. Note that you will be using $k = 2$ in your spectral clustering algorithm.

# Problem 3: Spectral Clustering vs. K-Means

### Part 1

Load the data from the two-dimenstional `two_moons.csv` and the three-dimensional `linked_chains.csv`. Each row represents a point. Note that the first column gives the ground truth - the cluster identity of each point - and so can be ignored. Plot each in 2d and 3d respectively, so you can see what is going on. 

### Part 2

Run K-Means on both. How do you do?

### Part 3

Find the best similarity function you can, and run Spectral Clustering. Which is better?

# Problem 4: More Spectral Clustering vs. K-Means

### Part 1

Generate an example where K-Means does better than Spectral Clustering. Explain how you constructed it and why, in addition to showing the plots.

# Problem 5: Computational Complexity and Spectral Clustering

### Part 1

This is a more open ended question. Computing the eigenvalues and eigenvectors of a matrix is not easy, and the complexity depends on the size of the matrix among other factors. An important factor is the sparsity of the matrix. Which aspects of your algorithm for spectral clustering could affect this? Using one of the examples from above, or another example of your choosing, try to illustrate this pheonomeon.